# Amazon S3 Fundamentals With Python Boto3 For Complete Beginners
## Learn how to use cloud resources in your Python scripts
![](images/pexels.jpg)

### Setup

In [4]:
import logging
import time
import warnings

import catboost as cb
import datatable as dt
import joblib
import lightgbm as lgbm
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import shap
import umap
import umap.plot
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.compose import *
from sklearn.impute import *
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import *

logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%d-%b-%y %H:%M:%S", level=logging.INFO
)
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings("ignore")
pd.set_option("float_format", "{:.5f}".format)

### Introduction

### Setup an account

Right, let's start with creating your AWS account, if you haven't already. Nothing unusual, just follow the steps from this [link](https://aws.amazon.com/free/?trk=ps_a131L0000085EJvQAM&trkCampaign=acq_paid_search_brand&sc_channel=ps&sc_campaign=acquisition_US&sc_publisher=google&sc_category=core-main&sc_country=US&sc_geo=NAMER&sc_outcome=acq&sc_detail=aws&sc_content=Brand_Core_aws_e&sc_segment=432339156150&sc_medium=ACQ-P|PS-GO|Brand|Desktop|SU|Core-Main|Core|US|EN|Text&s_kwcid=AL!4422!3!432339156150!e!!g!!aws&ef_id=EAIaIQobChMIxa2ogpvA9QIVxP7jBx2iAwgNEAAYASAAEgJGR_D_BwE:G:s&s_kwcid=AL!4422!3!432339156150!e!!g!!aws&all-free-tier.sort-by=item.additionalFields.SortRank&all-free-tier.sort-order=asc&awsf.Free%20Tier%20Types=*all&awsf.Free%20Tier%20Categories=*all):

![](images/sign_up.gif)

Then, we will go to the [AWS IAM (Identity and Access Management) console](https://console.aws.amazon.com/), which is the place we will be doing most of the work.

![](images/aws_console.gif)

From the console, you can easily switch between different AWS servers, create users, add policies and allow access to your user account. We will do each one by one.

### Create a user

For one AWS account, you can create multiple users and each user can have various levels of access to your account's resources. Let's create a sample user for this tutorial:

![](images/create_user.gif)

In the IAM console:

1. Go to the users tab.
2. Click on Add users.
3. Enter a username in the field.
4. Tick the "Access key - Programmatic access field" (essential).
5. Click "Next" and "Attach existing policies directly".
6. Tick the "AdministratorAccess" policy.
7. Click "Next" until you see the "Create user" button
8. Finally, download the given CSV file of your user's credentials.

It should look like this:

![](images/credentials.png)

Store it somewhere safe, because we will be using the credentials later. 

### Create a bucket

Now, let's create an S3 bucket where we can store data.

![](images/create_bucket.gif)

In the [IAM console](https://console.aws.amazon.com/):

1. Click services in the top left corner.
2. Scroll down to storage and select S3 from the right-hand list.
3. Click "Create bucket" and give it a name. 

You can choose any region you want. Leave the rest of the settings as is and click "Create bucket" once more. 

### Create a policy and add it to your user

In AWS, access is management through policies. A policy can be a set of settings or a JSON file attached to an AWS object (user, resource, group, roles) and it controls what aspects of the object you can use. 

Below, we will create a policy that enables us to interact with our bucket programmatically - i.e., through the CLI or in a script. 

![](images/create_policy.gif)

In the [IAM console](https://console.aws.amazon.com/):

1. Go to the Policies tab and click "Create a policy".
2. Click the "JSON" tab and insert the code below:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "ConsoleAccess",
            "Effect": "Allow",
            "Action": [
                "s3:*"
            ],
            "Resource": [
                "arn:aws:s3:::your-bucket-name",
                "arn:aws:s3:::your-bucket-name/*"
            ]
        }
    ]
}
```
replacing *your-bucket-name* with your own. If you pay attention, in the Action field of the JSON, we are putting `s3:*` to allow any type of interaction to our bucket. This is very broad, so you may allow only certain actions. In that case, check out [this page](https://docs.aws.amazon.com/IAM/latest/UserGuide/reference_policies_examples_s3_rw-bucket-console.html) of the AWS docs to learn how limit access.

Now, this policy is only attached to the bucket. We should attach it to the user as well so that your API credentials work properly. Here are the instructions:

![](images/add_permission.gif)

In the [IAM console](https://console.aws.amazon.com/):

1. Go the Users tab and click on the user we created in the last section.
2. Click the "Add permissions" button.
3. Click "Attach existing policies" tab.
4. Filter them by the policy we just created.
5. Tick the policy, review it and click "Add" the final time.

### Download AWS CLI and configure your user

Now, we download the AWS command-line tool, because it makes authentication so much easier. Kindly go to [this page](https://aws.amazon.com/cli/) and download the executable for your platform:

![](images/download_cli.gif)

Run the executable and reopen any active terminal sessions to let the changes take effect. Then, type `aws configure`:

![](images/configure_aws.gif)

Insert your AWS Key ID and Secret Access Key, along with the region your created your bucket in (use the CSV file). You can find the region name of your bucket in the S3 page of the console:

![](images/region.png)

Just click "Enter" when you reach the Default Output Format field in the configuration. There won't be any output.

### Upload your files and copy URI

![](images/upload_file.gif)

![](images/copy_uri.gif)

### Using Python Boto3 to download files from S3 bucket

### Other operations with Boto3